In [1]:
#This File is for assigning The clustering priority to the products based on the cluster they belong to.

# clusters Given as 
# Included Their MainClusters/ClusterLabels >> { "Same-Day Urgent",""Next-Day Urgent","Mid-Term","Low Priority",""Unknown"" } 

#  SubClusters/Handling_Group
# { "Normal", "Fragile+Temp", "Temp Only", "Fragile Only" }
# Normal          2423
# Fragile+Temp    1203
# Temp Only        594
# Fragile Only     586

#KNN Clusters 
# {0 to 5}

# FILE PATHS FLOW

# AAALLLLLL paths = {
#     "products": r"C:\Users\Administrator\Desktop\Python\ML\DataSets\RawDataSets\products\products.csv",
#     "assignments": r"C:\Users\Administrator\Desktop\Python\ML\DataSets\RawDataSets\transport_assignments\transport_assignments.csv",
    # "dispatches": r"C:\Users\Administrator\Desktop\Python\ML\DataSets\RawDataSets\dispatches\dispatches.csv"
# }

#MAIN CLUSTER
input_path = r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\cluster_input_master.csv"

# GIVESS >>>>

output_path = r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\cluster1_main_grouped.csv"

#SUB CLUSTER/TEMP, FRAG...
input_path = r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\cluster1_main_grouped.csv"

# GIVESS >>>>

output_path = r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\subcluster_handling_output.csv"



#TRANSPORT VEHICLE DATASETS

#That are bsaed upon the vehicle ID and divides into its respective vehicle type as General/Specialised etc
vehicle_type_map = "C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\vehicle_type_map.csv"

#That are bsaed upon the vehicle ID and Refrer into its vehicle type, CapacityType , volume capacity, weight capacity 
vehicle_type_map_with_capacity = "C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\vehicle_type_map_with_capacity.csv"


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1879892766.py, line 43)

In [1]:

# Output path

import pandas as pd

# File paths
paths = {
    "products": r"C:\Users\Administrator\Desktop\Python\ML\DataSets\RawDataSets\products\products.csv",
    "assignments": r"C:\Users\Administrator\Desktop\Python\ML\DataSets\RawDataSets\transport_assignments\transport_assignments.csv",
    "dispatches": r"C:\Users\Administrator\Desktop\Python\ML\DataSets\RawDataSets\dispatches\dispatches.csv"
}

output_path = r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\cluster_input_master.csv"


# Load datasets
df_products = pd.read_csv(paths["products"])
df_assignments = pd.read_csv(paths["assignments"])
df_dispatches = pd.read_csv(paths["dispatches"])

# Select columns
products_cols = [
    "Product_ID", "Unit_Weight (kg)", "Unit_Volume (L)", "Fragile_Flag", "Temp_Sensitive_Flag"
]

assignments_cols = [
    "Assignment_ID", "Product_ID", "Urgent_Flag", "Dispatch_Window", "Delivery_Window"
]

dispatches_cols = [
    "Assignment_ID", "Quantity_Dispatched"
]

# Apply selection
df_products = df_products[products_cols]
df_assignments = df_assignments[assignments_cols]
df_dispatches = df_dispatches[dispatches_cols].rename(columns={"Quantity_Dispatched": "Quantity_Assigned"})

# Merge all 3 datasets
df_merge_1 = pd.merge(df_assignments, df_dispatches, on="Assignment_ID", how="left")
df_final = pd.merge(df_merge_1, df_products, on="Product_ID", how="left")

# Derived metrics
df_final["Total_Weight"] = df_final["Unit_Weight (kg)"] * df_final["Quantity_Assigned"]
df_final["Total_Volume"] = df_final["Unit_Volume (L)"] * df_final["Quantity_Assigned"]

# Blank fields for ML model outputs
df_final["Priority_Label"] = ""
df_final["Cluster_Label"] = ""
df_final["Vehicle_ID"] = ""

# Final column order
final_columns = [
    "Assignment_ID", "Product_ID", "Urgent_Flag", "Dispatch_Window", "Delivery_Window",
    "Unit_Weight (kg)", "Unit_Volume (L)", "Fragile_Flag", "Temp_Sensitive_Flag",
    "Quantity_Assigned", "Total_Weight", "Total_Volume",
    "Priority_Label", "Cluster_Label", "Vehicle_ID"
]

df_final = df_final[final_columns]

# Export
df_final.to_csv(output_path, index=False)
print(f"✅ Master clustering CSV generated:\n{output_path}")



✅ Master clustering CSV generated:
C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\cluster_input_master.csv


In [3]:
# CLUSTER 1 // MAIN CLUSTER

# --- Cell 1: Imports ---
import pandas as pd
from datetime import datetime

# --- Cell 2: Load the Master CSV ---
input_path = r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\cluster_input_master.csv"
df = pd.read_csv(input_path)

# --- Cell 3: Helper Function to Assign Main Cluster Label ---
def assign_main_cluster(row):
    try:
        delivery_days_left = (pd.to_datetime(row["Delivery_Window"]) - pd.Timestamp.today()).days
        urgent = row.get("Urgent_Flag", 0)

        if urgent == 1 and delivery_days_left <= 1:
            return "Same-Day Urgent"
        elif urgent == 1 and delivery_days_left <= 3:
            return "Next-Day Urgent"
        elif delivery_days_left <= 5:
            return "Mid-Term"
        else:
            return "Low Priority"
    except:
        return "Unknown"

# --- Cell 4: Apply Main Cluster ---
df["Main_Cluster_Label"] = df.apply(assign_main_cluster, axis=1)

# --- Cell 5: Save the output ---
output_path = r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\cluster1_main_grouped.csv"
df.to_csv(output_path, index=False)
print(f"✅ Main cluster tags added and saved to:\n{output_path}")


✅ Main cluster tags added and saved to:
C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\cluster1_main_grouped.csv


In [5]:
# CLUSTER 2 // SUB CLUSTER

import pandas as pd

# Load the master dataset
input_path = r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\cluster1_main_grouped.csv"
df = pd.read_csv(input_path)

# Sanitize: Ensure boolean flags are integers (0 or 1)
df["Fragile_Flag"] = df["Fragile_Flag"].astype(int)
df["Temp_Sensitive_Flag"] = df["Temp_Sensitive_Flag"].astype(int)

# Define sub-cluster group function
def assign_handling_group(row):
    if row["Fragile_Flag"] == 1 and row["Temp_Sensitive_Flag"] == 1:
        return "Fragile+Temp"
    elif row["Fragile_Flag"] == 1 and row["Temp_Sensitive_Flag"] == 0:
        return "Fragile Only"
    elif row["Fragile_Flag"] == 0 and row["Temp_Sensitive_Flag"] == 1:
        return "Temp Only"
    else:
        return "Normal"

# Apply sub-clustering label
df["Handling_Group"] = df.apply(assign_handling_group, axis=1)

# Save to updated file
output_path = r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\subcluster_handling_output.csv"
df.to_csv(output_path, index=False)

print(f"✅ Sub-clustering done based on handling requirements.")
print(f"📦 Output saved to:\n{output_path}")
print("📊 Distribution:\n", df["Handling_Group"].value_counts())


✅ Sub-clustering done based on handling requirements.
📦 Output saved to:
C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\subcluster_handling_output.csv
📊 Distribution:
 Handling_Group
Normal          2423
Fragile+Temp    1203
Temp Only        594
Fragile Only     586
Name: count, dtype: int64


In [6]:
#GENRATES SPECIAL TRANSPORTERS

import pandas as pd

# Load transport assignments
assignment_path = r"C:\Users\Administrator\Desktop\Python\ML\DataSets\RawDataSets\transport_assignments\transport_assignments.csv"
df = pd.read_csv(assignment_path)

# Helper to extract Vehicle_ID from Assignment_ID
def extract_vehicle_id(assignment_id):
    try:
        return assignment_id.split("::")[3].strip()
    except:
        return "UNKNOWN"

# Helper to extract Vehicle_Type from Transport_ID
def extract_vehicle_type(transport_id):
    try:
        return transport_id.split("::")[4].strip()
    except:
        return "UNKNOWN"

# Apply extractors
df["Vehicle_ID"] = df["Assignment_ID"].apply(extract_vehicle_id)
df["Vehicle_Type"] = df["Transport_ID"].apply(extract_vehicle_type)

# Drop duplicates to get unique vehicle-to-type mapping
vehicle_df = df[["Vehicle_ID", "Vehicle_Type"]].drop_duplicates().reset_index(drop=True)

# Export to CSV
output_path = r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\vehicle_type_map.csv"
vehicle_df.to_csv(output_path, index=False)

print(f"✅ Vehicle map file saved to:\n{output_path}")
print(vehicle_df.head())


✅ Vehicle map file saved to:
C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\vehicle_type_map.csv
  Vehicle_ID Vehicle_Type
0  GJ08X2709      General
1  GJ30X2371  Specialised
2  GJ24X8239      General
3  GJ01X1573  Specialised
4  GJ20X7043      General


In [ ]:
#GETTING VEHICLE CAPACITY AND WEIGHT + VOLUME 

import pandas as pd

# Input path
input_path = r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\vehicle_type_map.csv"

# Load vehicle type map
df = pd.read_csv(input_path)

# Define mapping dict for vehicle types
capacity_mapping = {
    "General":        ("Truck",        35000, 9000),
    "Specialised":    ("Sp.Van",       15000, 4000),
    "Mini-Truck":     ("MiniTruck",    10000, 2500),
    "Van":            ("Van",           7000, 1800),
    "Bike":           ("TwoWheeler",     250,   50),
    "Small Van":      ("SmallVan",      5000, 1200),
}

# Apply mapping
df["Capacity_Type"] = df["Vehicle_Type"].apply(lambda x: capacity_mapping.get(x, ("Unknown", 0, 0))[0])
df["Volume_Capacity (L)"] = df["Vehicle_Type"].apply(lambda x: capacity_mapping.get(x, ("Unknown", 0, 0))[1])
df["Weight_Capacity (kg)"] = df["Vehicle_Type"].apply(lambda x: capacity_mapping.get(x, ("Unknown", 0, 0))[2])

# Output path
output_path = r"C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\vehicle_type_map.csv"
df.to_csv(output_path, index=False)

print(f"✅ Extended vehicle data saved at:\n{output_path}")
print(df.head())


✅ Extended vehicle data saved at:
C:\Users\Administrator\Desktop\Python\ML\DataSets\projectplancluster\vehicle_type_map_with_capacity.csv
  Vehicle_ID Vehicle_Type Capacity_Type  Volume_Capacity (L)  \
0  GJ08X2709      General         Truck                35000   
1  GJ30X2371  Specialised        Sp.Van                15000   
2  GJ24X8239      General         Truck                35000   
3  GJ01X1573  Specialised        Sp.Van                15000   
4  GJ20X7043      General         Truck                35000   

   Weight_Capacity (kg)  
0                  9000  
1                  4000  
2                  9000  
3                  4000  
4                  9000  
